<a id="readme-top"></a>

# LP read mps notebook

## 📚 Table of Content
- [Reading File to String](#reading-file-to-string)
- [Reading MPS File From String](#reading-mps-file-from-string)
    - [Original Function](#original-function)
    - [Adding Multiple Column Support](#adding-multiple-column-support)
    - [Adding MIP Support](#adding-mip-support)
- [Unit Testing](#unit-testing)
    - [Manual Approach](#manual-approach)
    - [Using JuMP MathOptInterface as a reference for testing](#using-jump-mathoptinterface-as-a-reference-for-testing)


## Discription Of MPS Format

## MPS File Format Documentation

The **MPS (Mathematical Programming System)** file format is a standardized format used for representing linear programming (LP) and mixed-integer programming (MIP) problems. Developed by IBM in the 1970s, it is widely used for exchanging problem instances between different optimization software.

### Structure of an MPS File

An MPS file is a plain text file that is divided into several sections. Each section begins with a keyword and is followed by relevant data:

#### 1. **NAME Section**
   - **Purpose**: Identifies the name of the problem.
   - **Format**: `NAME <problem_name>`

#### 2. **ROWS Section**
   - **Purpose**: Defines the objective function and constraints.
   - **Format**: Each row is identified by a type and name.
   - **Row Types**:
     - `N`: The objective function.
     - `L`: Less than or equal to (≤) constraints.
     - `G`: Greater than or equal to (≥) constraints.
     - `E`: Equality (=) constraints.

#### 3. **COLUMNS Section**
   - **Purpose**: Specifies the coefficients of variables in the objective function and constraints.
   - **Format**: Each line in this section specifies a variable and its coefficient in one or two constraints or the objective function.
   - **Example**:
     ```
     COLUMN_NAME ROW_NAME coefficient
     ```

#### 4. **RHS Section**
   - **Purpose**: Specifies the right-hand side (RHS) values for the constraints.
   - **Format**: Similar to the COLUMNS section but with RHS values.

#### 5. **BOUNDS Section**
   - **Purpose**: Specifies the bounds on the variables.
   - **Types of Bounds**:
     - `LO`: Lower bound.
     - `UP`: Upper bound.
     - `FX`: Fixed variable (both lower and upper bound are equal).
     - `FR`: Free variable (no bounds).
   - **Integer Variable Definition Using Markers**
   In addition to the standard bound types, some solvers use marker tags to define integer variables. The `MARKER 'INTORG' 'MARKER'` and `MARKER 'INTEND' 'MARKER'` approach is used to indicate blocks of variables that should be treated as integers.



#### Example Usage:

```mps
MARKER 'INTORG' 'MARKER'
    X1    ROW1    1
    X2    ROW2    2
    X3    ROW3    3
MARKER 'INTEND' 'MARKER'
   - **Format**: 
     ```
     BOUND_TYPE BOUND_NAME VARIABLE_NAME value
     ```

### 6. **ENDATA Section**
   - **Purpose**: Marks the end of the MPS file.
   - **Format**: The keyword `ENDATA`.

## Example MPS File

## References
<sup id="cite2">Gurobi Optimization, 2020</sup>

- [MPS Format - Gurobi Documentation](https://www.gurobi.com/documentation/9.1/refman/mps_format.html)
- [MPS File Format - IBM Documentation](https://www.ibm.com/docs/en/icos/12.9.0?topic=formats-mps-file-format)




In [ ]:
using LinearAlgebra
using SparseArrays
using Random
using ArgParse
using DataStructures
using Test

# JuMP
using JuMP
using MathOptInterface
const MOI = MathOptInterface

# local modules
push!(LOAD_PATH, realpath("../code"))
using lp_constants
using lp_utils
using lp_problem


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Reading File to String

The following function reads a file and outputs a string. It is an important step because we can use string methods and technequies to access data.

In [ ]:
function read_file_to_string(file_path::String)
    file_string = open(file_path, "r") do f
        read(f, String)
    end
    return file_string
end

<p align="right">(<a href="#readme-top">back to top</a>)</p>

<details>
<summary><h3>Original Function</h3></summary>

```julia
function read_mps_from_string_multicolumns(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    objective_set = false
    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, and FX
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR
                bound_type, _, var_name = words
                value = Inf
            end
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    return LPProblem(is_minimize, c, A, b, lb, ub, vars, constraint_types)
end
</details> ```

<details>
<summary><h3>Adding Multiple Column Support</h3></summary>

```julia
function read_mps_from_string_multicolumns(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    objective_set = false
    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, and FX
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR
                bound_type, _, var_name = words
                value = Inf
            end
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    return LPProblem(is_minimize, c, A, b, lb, ub, vars, constraint_types)
end
```
</details> 

<details>
    <summary> Other code</summary>

```julia
function read_mps_from_string_multicolumns(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    objective_set = false
    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, and FX
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR
                bound_type, _, var_name = words
                value = Inf
            end
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    return LPProblem(is_minimize, c, A, b, lb, ub, vars, constraint_types)
end
```
</details>

<details>
    <summary> Test </summary>

```julia
mps_filename = "../../benchmarks/mps_files/test.mps"
test_string = read_file_to_string(mps_filename)

lp = read_mps_from_string_multicolumns(test_string)
println(lp)

println("")

dense_lp = Matrix(lp.A)
println("c: ", lp.c)
println("b: ", lp.b)
println("vars: ", lp.vars)
println("constraint_types: ", lp.constraint_types)
println("A: ", dense_lp)
```
</details>

<p align="right">(<a href="#readme-top">back to top</a>)</p>

### Adding MIP Support

#### New MIP Struct

In [ ]:
# struct MIPProblem
#     is_minimize::Bool  # True if the objective is to minimize
#     c::Vector{Float64}  # Objective function coefficients
#     A::SparseMatrixCSC{Float64, Int64}  # Constraint matrix
#     b::Vector{Float64}  # Right-hand side of constraints
#     l::Vector{Float64}  # Variable lower bounds
#     u::Vector{Float64}  # Variable upper bounds
#     vars::Vector{String}  # Variable names
#     variable_types::Vector{Symbol}  # Array of variable types, same length as vars
#     constraint_types::Vector{Char}  # Constraint types
# end

In [ ]:
MPS_MIP_Example = """
NAME          TESTPROB
ROWS
 N  OBJ
 L  ROW1
 G  ROW2
COLUMNS
    X1    OBJ     1
    X1    ROW1    2
    X2    OBJ     1
    X2    ROW2    1
    MARKER 'INTORG' 'MARKER'
    X3    OBJ     1
    X3    ROW1    1
    MARKER 'INTEND' 'MARKER'
RHS
    RHS1   ROW1    5
    RHS1   ROW2    10
BOUNDS
 UP BND1  X1      10
 LO BND1  X1      0
 UP BND1  X2      1
 LO BND1  X2      0
 UP BND1  X3      15
 LO BND1  X3      0
ENDATA
"""


<details>
    <summary> First function </summary>

```julia 

function read_mps_from_string_mip(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true
    objective_set = false
    in_integer_block = false

    # New array to keep track of variable types (Continuous by default)
    variable_types = OrderedDict{String, Symbol}()

    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            if words[1] == "MARKER"
                if words[2] == "'INTORG'"
                    in_integer_block = true
                elseif words[2] == "'INTEND'"
                    in_integer_block = false
                end
                continue
            end

            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
                # Set variable type based on whether we are in an integer block
                variable_types[col_name] = in_integer_block ? :Integer : :Continuous
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, FX, BV, LI, UI, SC, SI
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR, MI, PL
                bound_type, _, var_name = words
                value = Inf
            end

            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end

            if bound_type in ["BV", "LI", "UI", "SC", "SI"]
                # Determine variable type based on the bound type
                if bound_type == "BV"
                    variable_types[var_name] = :Binary
                elseif bound_type == "LI" || bound_type == "UI"
                    variable_types[var_name] = :Integer
                elseif bound_type == "SC"
                    variable_types[var_name] = :SemiContinuous
                elseif bound_type == "SI"
                    variable_types[var_name] = :SemiInteger
                end
            end

            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to MIPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    # Convert variable_types from OrderedDict to an array
    variable_types_array = [variable_types[var] for var in vars]

    # Construct the MIPProblem struct
    mip_problem = MIPProblem(
        is_minimize,
        c,
        A,
        b,
        lb,
        ub,
        vars,
        variable_types_array,  # Array of variable types
        constraint_types
    )

    return mip_problem
end
```
</details>

In [ ]:
"""
    read_mps_from_string_mip(mps_string::String) -> MIPProblem

This function parses a given MPS (Mathematical Programming System) formatted string and converts it into a `MIPProblem` struct, which is used to represent a Mixed Integer Programming (MIP) problem in Julia.

# Arguments:
- `mps_string::String`: A string representing the content of an MPS file. The MPS format is a standard input format for mathematical programming problems, including linear programming (LP) and mixed-integer programming (MIP).

# Returns:
- `MIPProblem`: A struct containing the following fields:
    - `is_minimize::Bool`: Indicates if the objective is to minimize (true) or maximize (false).
    - `c::Vector{Float64}`: The coefficients of the objective function.
    - `A::SparseMatrixCSC{Float64, Int64}`: The constraint matrix in Compressed Sparse Column (CSC) format.
    - `b::Vector{Float64}`: The right-hand side vector of the constraints.
    - `l::Vector{Float64}`: The lower bounds of the variables.
    - `u::Vector{Float64}`: The upper bounds of the variables.
    - `vars::Vector{String}`: The names of the variables.
    - `variable_types::Vector{Symbol}`: An array indicating the type of each variable (e.g., :Continuous, :Integer, :Binary).
    - `constraint_types::Vector{Char}`: An array indicating the type of each constraint ('L' for ≤, 'G' for ≥, 'E' for =).

# Notes:
- The function handles both continuous and integer variables, as well as binary variables and various bound types, such as fixed variables and semi-continuous variables.
- The MPS format is case-insensitive, and the function is designed to skip comments and empty lines.
- Constraint types are adjusted for 'G' constraints by multiplying the corresponding rows in the matrix and the right-hand side by -1.

This function is useful for converting MPS files into a structured format suitable for further processing and solving in Julia's optimization frameworks.
"""


function read_mps_from_string_mip(mps_string::String)
    # Split the input MPS string into individual lines
    lines = split(mps_string, '\n')

    # Initialize a dictionary to store different sections of the MPS file
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())

    # Variables to keep track of the current section, objective name, and optimization direction
    current_section = ""
    objective_name = ""
    is_minimize = true  # Default to minimizing unless specified otherwise
    objective_set = false  # Flag to check if objective sense has been set

    # Flags to track whether we are in an integer block for variable types
    in_integer_block = false

    # New array to keep track of variable types (Continuous by default)
    variable_types = OrderedDict{String, Symbol}()

    # Loop through each line of the MPS file
    for line in lines
        # Split the line into words (tokens)
        words = split(line)
        
        # Skip empty lines or comment lines (starting with '*')
        (isempty(words) || (line[1] == '*')) && continue

        # Check for section headers and update the current section
        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        # Handle each section based on the current_section value
        if current_section == "NAME"
            # Store the name of the MPS file (first word after "NAME")
            sections["NAME"] = words[1]

        elseif current_section == "OBJSENSE"
            # Set the optimization direction (minimize or maximize)
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end

        elseif current_section == "ROWS"
            # Store each row's type and name in the "ROWS" section
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            
            # Identify the objective function row (marked with 'N')
            if row_type == "N"
                objective_name = row_name
            end

        elseif current_section == "COLUMNS"
            # Handle integer block markers to distinguish variable types
            if words[1] == "MARKER"
                if words[2] == "'INTORG'"
                    in_integer_block = true
                elseif words[2] == "'INTEND'"
                    in_integer_block = false
                end
                continue  # Skip processing the marker line itself
            end

            # Process the column (variable) name and associated rows/values
            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
                # Set variable type based on whether we are in an integer block
                variable_types[col_name] = in_integer_block ? :Integer : :Continuous
            end

            # Store values for the first row associated with this column
            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            # If a second row/value pair exists, store that as well
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end

        elseif current_section == "RHS"
            # Handle the RHS (right-hand side) values
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)

        elseif current_section == "BOUNDS"
            # Handle variable bounds and update their types if needed
            if length(words) == 4  # Bound types with a specific value (e.g., LO, UP, FX, BV)
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # Bound types without a specific value (e.g., FR, MI, PL)
                bound_type, _, var_name = words
                value = Inf  # Default to infinity if no value is specified
            end

            # Initialize bounds for the variable if not already present
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end

            # Determine variable type based on the bound type
            if bound_type in ["BV", "LI", "UI", "SC", "SI"]
                if bound_type == "BV"
                    variable_types[var_name] = :Binary
                elseif bound_type == "LI" || bound_type == "UI"
                    variable_types[var_name] = :Integer
                elseif bound_type == "SC"
                    variable_types[var_name] = :SemiContinuous
                elseif bound_type == "SI"
                    variable_types[var_name] = :SemiInteger
                end
            end

            # Store the bound value in the appropriate dictionary entry
            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing  # Free variable, no specific bounds
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to MIPProblem structure
    vars = collect(keys(sections["COLUMNS"]))  # List of variables
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    # Initialize the objective coefficients, constraint matrix, RHS, and constraint types
    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate the objective function coefficients from the COLUMNS section
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate the constraint matrix and RHS vector
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"  # Skip the objective row
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store the type of the constraint
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust the constraint if it is of type 'G' (≥)
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints to populate lower and upper bounds
    lb = fill(-Inf, n_vars)  # Initialize lower bounds to -Inf
    ub = fill(Inf, n_vars)   # Initialize upper bounds to Inf
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]  # Fixed variable (lower and upper bounds are equal)
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    # Convert variable_types from OrderedDict to an array for easier use in the struct
    variable_types_array = [variable_types[var] for var in vars]

    # Construct the MIPProblem struct with all the parsed data
    mip_problem = MIPProblem(
        is_minimize,            # Optimization direction
        c,                      # Objective function coefficients
        A,                      # Constraint matrix
        b,                      # Right-hand side of constraints
        lb,                     # Variable lower bounds
        ub,                     # Variable upper bounds
        vars,                   # Variable names
        variable_types_array,   # Array of variable types
        constraint_types        # Constraint types
    )

    # Return the constructed MIPProblem
    return mip_problem
end

In [ ]:
mps_string = read_file_to_string("../../benchmarks/mps_files/blend.mps")
println(mps_string)

In [ ]:
#mps_string = MPS_MIP_Example

mps_string = read_file_to_string("../../benchmarks/mps_files/blend.mps")
lp_mip = read_mps_from_string_mip(mps_string)

println("Objective function coefficients: ", lp_mip.c)
println("Constraint matrix: ", lp_mip.A)
println("Right-hand side: ", lp_mip.b)
println("Lower bounds: ", lp_mip.l)
println("Upper bounds: ", lp_mip.u)
println("Variable types: ", lp_mip.variable_types)
println("constraint types types: ", lp_mip.constraint_types)


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Unit Testing

### Manual Approach

In [ ]:
mps_filename = "../../benchmarks/mps_files/test.mps"
lp_string = read_file_to_string(mps_filename)
println(lp_string)

In [ ]:
# # Define the expected outputs for comparison
# expected_c = [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0]
# expected_b = [200.0, 250.0, 0.0, 0.0, 0.0]
# expected_vars = ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"]
# expected_constraint_types = ['L', 'L', 'L', 'G', 'E']

# # Define the expected A matrix (as a sparse matrix for comparison)
# expected_A = [
#     [1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
#     [0.0, 0.0, 1.0, 1.0, 1.0, 0.0],
#     [8.8, 6.1, 2.0, 4.2, 5.0, -6.0],
#     [8.8, 6.1, 2.0, 4.2, 5.0, -3.0],
#     [1.0, 1.0, 1.0, 1.0, 1.0, -1.0]
# ]

# # The map(x -> x == -0.0 ? 0.0 : x, lp.) is used to ensure no non zeros.

# # Write the test cases
# @testset "MPS Parsing Tests" begin
#     # Test the objective function coefficients
#     @test map(x -> x == -0.0 ? 0.0 : x, lp.c) == expected_c
    
#     # Test the right-hand side vector
#     @test map(x -> x == -0.0 ? 0.0 : x, lp.b) == expected_b
    
#     # Test the variable names
#     @test map(x -> x == -0.0 ? 0.0 : x, lp.vars) == expected_vars
    
#     # Test the constraint types
#     @test map(x -> x == -0.0 ? 0.0 : x, lp.constraint_types) == expected_constraint_types
    
#     # Test the constraint matrix (A)
#     #@test Matrix(lp.A) == expected_A # This indicates an issue.
# end

### Using JuMP MathOptInterface as a reference for testing

<details>
    <summary> Read LP with Jump </summary>

```julia
"""
    read_and_process_mps(file_path::String) -> LPProblem

Reads an MPS file from the provided file path, processes it into a MathOptInterface model, 
and returns an LPProblem struct with all the relevant details.

# Arguments
- `file_path::String`: The file path to the MPS file.

# Returns
- `LPProblem`: A struct representing the linear programming problem.
"""
function read_mps_with_JuMP(file_path::String)
    # Create a utility model
    model = MOI.Utilities.Model{Float64}()

    # Read the MPS file into the model
    MOI.read_from_file(model, file_path)

    # Extract variables
    variables = MOI.get(model, MOI.ListOfVariableIndices())

    # Extract variable names
    variable_names = [MOI.get(model, MOI.VariableName(), var) for var in variables]

    # Extract the objective function (assumes a linear objective)
    objective_function = MOI.get(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}())
    objective_coeffs = zeros(Float64, length(variables))

    # Populate the objective coefficients array
    for term in objective_function.terms
        objective_coeffs[term.variable.value] = term.coefficient
    end

    # Determine whether it's a minimization or maximization problem
    is_minimize = MOI.get(model, MOI.ObjectiveSense()) == MOI.MIN_SENSE

    # Separate constraints by type
    less_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())
    greater_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.GreaterThan{Float64}}())
    equal_to_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.EqualTo{Float64}}())

    constraint_matrix_rows = Int64[]
    constraint_matrix_cols = Int64[]
    constraint_matrix_vals = Float64[]
    rhs_values = Float64[]
    constraint_types = Char[]

    # Process LessThan constraints
    for con in less_than_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.upper)
        push!(constraint_types, 'L')
    end

    # Process GreaterThan constraints
    for con in greater_than_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.lower)
        push!(constraint_types, 'G')
    end

    # Process EqualTo constraints
    for con in equal_to_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.value)
        push!(constraint_types, 'E')
    end

    # Convert to sparse matrix
    constraint_matrix = sparse(constraint_matrix_rows, constraint_matrix_cols, constraint_matrix_vals, length(rhs_values), length(variables))

    # Define lower and upper bounds
    lower_bounds = fill(0.0, length(variables))  # Default lower bounds (0.0)
    upper_bounds = fill(Inf, length(variables))  # Default upper bounds (Inf)

    # Construct the LPProblem struct
    lp = LPProblem(
        is_minimize,
        objective_coeffs,
        constraint_matrix,
        rhs_values,
        lower_bounds,
        upper_bounds,
        variable_names,
        constraint_types
    )

    return lp
end
```
</details>

In [ ]:
# Function to determine the type of variable
function get_variable_type(var)
    if is_binary(var)
        return "Binary"
    elseif is_integer(var)
        return "Integer"
    else
        return "Continuous"
    end
end


In [ ]:
"""
    read_and_process_mps(file_path::String) -> LPProblem

Reads an MPS file from the provided file path, processes it into a MathOptInterface model, 
and returns an LPProblem struct with all the relevant details.

# Arguments
- `file_path::String`: The file path to the MPS file.

# Returns
- `MIPProblem`: A struct representing the linear programming problem.
"""
function read_mps_with_JuMP_MIP(file_path::String)
    # Create a JuMP model
    model = Model()

    # Read the MPS file into the model
    MOI.read_from_file(model.moi_backend, file_path)

    # Extract variables
    variables = all_variables(model)

    # Extract variable names
    variable_names = [name(var) for var in variables]

    # Extract the objective function (assumes a linear objective)
    objective_function = MOI.get(model.moi_backend, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}())
    objective_coeffs = zeros(Float64, length(variables))

    # Populate the objective coefficients array
    for term in objective_function.terms
        objective_coeffs[term.variable.value] = term.coefficient
    end

    # Determine whether it's a minimization or maximization problem
    is_minimize = MOI.get(model.moi_backend, MOI.ObjectiveSense()) == MOI.MIN_SENSE

    # Separate constraints by type
    less_than_constraints = MOI.get(model.moi_backend, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())
    greater_than_constraints = MOI.get(model.moi_backend, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.GreaterThan{Float64}}())
    equal_to_constraints = MOI.get(model.moi_backend, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.EqualTo{Float64}}())

    constraint_matrix_rows = Int64[]
    constraint_matrix_cols = Int64[]
    constraint_matrix_vals = Float64[]
    rhs_values = Float64[]
    constraint_types = Char[]

    # Process LessThan constraints
    for con in less_than_constraints
        func = MOI.get(model.moi_backend, MOI.ConstraintFunction(), con)
        set = MOI.get(model.moi_backend, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.upper)
        push!(constraint_types, 'L')
    end

    # Process GreaterThan constraints
    for con in greater_than_constraints
        func = MOI.get(model.moi_backend, MOI.ConstraintFunction(), con)
        set = MOI.get(model.moi_backend, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.lower)
        push!(constraint_types, 'G')
    end

    # Process EqualTo constraints
    for con in equal_to_constraints
        func = MOI.get(model.moi_backend, MOI.ConstraintFunction(), con)
        set = MOI.get(model.moi_backend, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.value)
        push!(constraint_types, 'E')
    end

    # Convert to sparse matrix
    constraint_matrix = sparse(constraint_matrix_rows, constraint_matrix_cols, constraint_matrix_vals, length(rhs_values), length(variables))

    # Define lower and upper bounds
    lower_bounds = fill(0.0, length(variables))  # Default lower bounds (0.0)
    upper_bounds = fill(Inf, length(variables))  # Default upper bounds (Inf)

    # Variable types array
    variable_types = Vector{Symbol}(undef, length(variables))

    # Determine the type of each variable using the provided get_variable_type function
    for (i, var) in enumerate(variables)
        variable_types[i] = Symbol(get_variable_type(var))
    end

    # Construct the MIPProblem struct
    mip_problem = MIPProblem(
        is_minimize,
        objective_coeffs,
        constraint_matrix,
        rhs_values,
        lower_bounds,
        upper_bounds,
        variable_names,
        variable_types,
        constraint_types,
    )

    return mip_problem
end


In [ ]:
read_mps_with_JuMP_MIP("../../benchmarks/mps_files/test.mps")

In [ ]:
# mps_filepath = "../../benchmarks/mps_files/simple.mps"
# test_mps_parsing_consistency(mps_filepath)


# # Run the test with a specific MPS file
# test_filepath = mps_filename
# test_read_mps_from_string_multicolumns(test_filepath)

# # 
# # mps_mip_filename = "../../benchmarks/mps_files/MIP-test.mps"
# # test_read_mps_from_string_multicolumns(mps_mip_filename)

## MPS Test Function

In [ ]:
function test_mps_parsing_consistency(mps_file::String; 
    check_is_minimize::Bool=true, 
    check_objective_coeffs::Bool=true, 
    check_constraint_matrix::Bool=true, 
    check_rhs_values::Bool=true, 
    check_lower_bounds::Bool=true, 
    check_upper_bounds::Bool=true, 
    check_variable_names::Bool=true, 
    check_constraint_types::Bool=true, 
    check_variable_types::Bool=true)
    @testset "Test consistency between MPS parsing functions with $mps_file" begin

        # Read the MPS file content as a string
        mps_string = read_file_to_string(mps_file)

        # Parse the MPS string using both functions
        lp_mip_1 = read_mps_from_string_mip(mps_string)
        lp_mip_2 = read_mps_with_JuMP_MIP(mps_file)

        # Conditional checks based on the arguments
        if check_is_minimize
            println("Testing is_minimize...")
            @test lp_mip_1.is_minimize == lp_mip_2.is_minimize || println("is_minimize check failed")
        end

        if check_objective_coeffs
            println("Testing objective coefficients...")
            @test lp_mip_1.c == lp_mip_2.c || println("Objective coefficients check failed")
        end

        if check_constraint_matrix
            println("Testing constraint matrix...")
            @test lp_mip_1.A == lp_mip_2.A || println("Constraint matrix check failed")
        end

        if check_rhs_values
            println("Testing RHS values...")
            @test lp_mip_1.b == lp_mip_2.b || println("RHS values check failed")
        end

        if check_lower_bounds
            println("Testing lower bounds...")
            @test lp_mip_1.l == lp_mip_2.l || println("Lower bounds check failed")
        end

        if check_upper_bounds
            println("Testing upper bounds...")
            @test lp_mip_1.u == lp_mip_2.u || println("Upper bounds check failed")
        end

        if check_variable_names
            println("Testing variable names...")
            @test lp_mip_1.vars == lp_mip_2.vars || println("Variable names check failed")
        end

        if check_constraint_types
            println("Testing constraint types...")
            @test lp_mip_1.constraint_types == lp_mip_2.constraint_types || println("Constraint types check failed")
        end

        if check_variable_types
            println("Testing variable types...")
            @test lp_mip_1.variable_types == lp_mip_2.variable_types || println("Variable types check failed")
        end

    end
end


In [ ]:
# Define the MPS file path
mps_filepath = "../../benchmarks/mps_files/simple.mps"

# Call the test function
test_mps_parsing_consistency(mps_filepath)

In [ ]:
# Define the MPS file path
mps_filepath = "../../benchmarks/mps_files/ex_9-7.mps"

# Call the test function
test_mps_parsing_consistency(mps_filepath, check_variable_types = false)

In [ ]:
# # # Define the MPS file path
# mps_filepath = "../../benchmarks/mps_files/test.mps"

# # Call the test function
# test_mps_parsing_consistency(mps_filepath, check_variable_types = false)

In [ ]:
mps_filepath = "../../benchmarks/mps_files/MIP-test.mps"
mps_string = read_file_to_string(mps_filepath)

println(read_mps_from_string_mip(mps_string))
#println(read_mps_with_JuMP_MIP(mps_filepath))
#lp_mip

In [ ]:
mps_filepath = "../../benchmarks/mps_files/problem.mps"
println(read_file_to_string(mps_filepath))

In [ ]:
# mps_filepath = "/Users/roryyarr/Desktop/Linear Programming/HiGHS/lp_code_other/HiGHS-repo/check/instances/shell.mps"
# mps_string = read_file_to_string(mps_filepath)

# # println(read_mps_from_string_mip(mps_string))
# # println(read_mps_with_JuMP_MIP(mps_filepath))

# # Call the test function
# test_mps_parsing_consistency(mps_filepath)#, check_variable_types = false)

In [ ]:
function read_mps_to_moi_model(mps_file::String)
    # Initialize a MathOptInterface model
    model = MOI.Utilities.Model{Float64}()
    
    # Load the MPS file into the model
    MOI.read_from_file(model, mps_file)

    return model
end

In [ ]:
mps_filepath = "../../benchmarks/mps_files/test.mps"
mip_string = read_file_to_string(mps_filepath)
println(mip_string)

In [ ]:
# Example usage:
mps_filepath = "../../benchmarks/mps_files/test.mps"
test = read_mps_to_moi_model(mps_filepath)
println(test)

In [ ]:
# # Example usage:
# mps_filepath = "../../benchmarks/mps_files/problem.mps"
# test = read_mps_to_moi_model(mps_filepath)
# println(test)

In [ ]:
# # Example usage:
# mps_filepath = "/Users/roryyarr/Desktop/Linear Programming/HiGHS/lp_code_other/HiGHS-repo/check/instances/shell.mps"
# test = read_mps_to_moi_model(mps_filepath)
# println(test)

In [ ]:
# # Example usage:
# mps_filepath = "/Users/roryyarr/Desktop/Linear Programming/HiGHS/lp_code_other/HiGHS-repo/check/instances/small_mip.mps"
# test = read_mps_to_moi_model(mps_filepath)
# println(test)

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## References

1. <a id="ref1"></a> Andersen, E.D., Andersen, K.D. Presolving in linear programming. *Mathematical Programming* 71, 221–245 (1995). [https://doi.org/10.1007/BF01586000](https://doi.org/10.1007/BF01586000)

2. <a id="ref2"></a> Gurobi Optimization, LLC. MPS Format. Gurobi Documentation (2020).https://www.gurobi.com/documentation/9.1/refman/mps_format.html